# Install dependencies.

In [1]:
!git clone https://github.com/openai/baselines.git
!git clone https://github.com/sunblaze-ucb/rl-generalization.git
  
# pip install git+https://github.com/openai/baselines

fatal: destination path 'baselines' already exists and is not an empty directory.
fatal: destination path 'rl-generalization' already exists and is not an empty directory.


In [2]:
!pip install tensorflow-gpu

In [3]:
import numpy as np

import tensorflow as tf
print(tf.test.gpu_device_name())

import sys
sys.path.append('rl-generalization')
sys.path.append('baselines')
import sunblaze_envs
import baselines

from baselines.common.tf_util import get_session

def clean_session_fix():
    get_session().close()
    tf.reset_default_graph() 

/device:GPU:0


In [0]:
import gym
from baselines import deepq

In [0]:
#%run baselines/baselines/deepq/experiments/train_pong.py

# Train and Play methods.

In [0]:
def callback(lcl, _glb):
    # stop training if reward exceeds 199
    is_solved = lcl['t'] > 100 and sum(lcl['episode_rewards'][-101:-1]) / 100 >= 199
    return is_solved


def train_cartpole(env, model_name):
    act = deepq.learn(
        env,
        network='mlp',
        lr=1e-3,
        total_timesteps=1000000,
        buffer_size=50000,
        exploration_fraction=0.1,
        exploration_final_eps=0.02,
        #checkpoint_path="cartpole",
        print_freq=100,
        callback=callback
    )
    
    print("Saving model to", model_name)
    act.save(model_name)
    
    return model_name
    
    
def play_cartpole(env, model_name):
    act = deepq.learn(env, network='mlp', total_timesteps=0, load_path=model_name)

    rewards = []
    for i in range(100):
        obs, done = env.reset(), False
        episode_rew = 0
        while not done:
            obs, rew, done, _ = env.step(act(obs[None])[0])
            episode_rew += rew
        rewards.append(episode_rew)
        
    return np.mean(rewards)

# Train on Deterministic. Test on ALL.

In [7]:
model_name = train_cartpole(env = sunblaze_envs.make('SunblazeCartPole-v0'), 
                            model_name = 'cartpole_model_d.pkl')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.


rl-generalization/sunblaze_envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Logging to /tmp/openai-2019-05-05-11-24-52-900074
--------------------------------------
| % time spent exploring  | 97       |
| episodes                | 100      |
| mean 100 episode reward | 21.6     |
| steps                   | 2134     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 200      |
| mean 100 episode reward | 24       |
| steps                   | 4536     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 300      |
| mean 100 episode reward | 24.2     |
| steps                   | 6954     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 91       |
| episodes                | 400      |
| mean 100 episode reward | 21.8     |
| steps                   | 9138     |
--------------------------------------
Saving model d

In [8]:
clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPole-v0'), model_name)
print('DD', avg_reward)

clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPoleRandomNormal-v0'), model_name)
print('DR', avg_reward)

clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPoleRandomExtreme-v0'), model_name)
print('DE', avg_reward)

rl-generalization/sunblaze_envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Loaded model from cartpole_model_d.pkl
DD 184.36
Loaded model from cartpole_model_d.pkl
DR 134.96
Loaded model from cartpole_model_d.pkl
DE 64.25


# Train on Stochastic. Test on ALL.

In [9]:
clean_session_fix()
model_name_r = train_cartpole(env = sunblaze_envs.make('SunblazeCartPoleRandomNormal-v0'), 
                              model_name = 'cartpole_model_r.pkl')

rl-generalization/sunblaze_envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 97       |
| episodes                | 100      |
| mean 100 episode reward | 23.3     |
| steps                   | 2306     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 200      |
| mean 100 episode reward | 24.4     |
| steps                   | 4741     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 300      |
| mean 100 episode reward | 23.6     |
| steps                   | 7104     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 400      |
| mean 100 episode reward | 26.1     |
| steps                   | 9711     |
--------------------------------------
Saving model due to mean reward increase: None -> 25.6
---------

In [10]:
clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPole-v0'), model_name_r)
print('RD', avg_reward)

clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPoleRandomNormal-v0'), model_name_r)
print('RR', avg_reward)

clean_session_fix()
avg_reward = play_cartpole(sunblaze_envs.make('SunblazeCartPoleRandomExtreme-v0'), model_name_r)
print('RE', avg_reward)

rl-generalization/sunblaze_envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Loaded model from cartpole_model_r.pkl
RD 200.0
Loaded model from cartpole_model_r.pkl
RR 200.0
Loaded model from cartpole_model_r.pkl
RE 172.32
